# L4: Training a Dual Encoder

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import numpy as np
import pandas as pd

from transformers import AutoTokenizer

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> file:</b> To access <code>requirements.txt</code> for this notebook, 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## The CrossEntropyLoss 'trick'

In [3]:
df = pd.DataFrame(
    [
        [4.3, 1.2, 0.05, 1.07],
        [0.18, 3.2, 0.09, 0.05],
        [0.85, 0.27, 2.2, 1.03],
        [0.23, 0.57, 0.12, 5.1]
    ]
)
data = torch.tensor(df.values, dtype=torch.float32)

In [4]:
def contrastive_loss(data):
    target = torch.arange(data.size(0))
    loss = torch.nn.CrossEntropyLoss()(data, target)
    return loss

# given the data, compute the loss given contrastive loss.

# cross enthropy --> softmax involved



In [5]:
torch.nn.Softmax(dim=1)(data)

tensor([[0.9100, 0.0410, 0.0130, 0.0360],
        [0.0429, 0.8801, 0.0393, 0.0377],
        [0.1512, 0.0846, 0.5832, 0.1810],
        [0.0075, 0.0105, 0.0067, 0.9753]])

In [6]:
torch.nn.Softmax(dim=1)(data).sum(dim=1)

tensor([1.0000, 1.0000, 1.0000, 1.0000])

In [7]:
N = data.size(0)
non_diag_mask = ~torch.eye(N, N, dtype=bool)

for inx in range(3):
    data = torch.tensor(df.values, dtype=torch.float32)
    data[range(N), range(N)] += inx*0.5
    # so you increase the value diagonally, but decrease the value elsewhere.
    # what happens is diagnol grows but the rest decrease. every iteration
    # loss keeps reducing every iteration.
    data[non_diag_mask] -= inx*0.02
    print(data)
    print(f"Loss = {contrastive_loss(data)}")


tensor([[4.3000, 1.2000, 0.0500, 1.0700],
        [0.1800, 3.2000, 0.0900, 0.0500],
        [0.8500, 0.2700, 2.2000, 1.0300],
        [0.2300, 0.5700, 0.1200, 5.1000]])
Loss = 0.19657586514949799
tensor([[4.8000, 1.1800, 0.0300, 1.0500],
        [0.1600, 3.7000, 0.0700, 0.0300],
        [0.8300, 0.2500, 2.7000, 1.0100],
        [0.2100, 0.5500, 0.1000, 5.6000]])
Loss = 0.12602083384990692
tensor([[5.3000, 1.1600, 0.0100, 1.0300],
        [0.1400, 4.2000, 0.0500, 0.0100],
        [0.8100, 0.2300, 3.2000, 0.9900],
        [0.1900, 0.5300, 0.0800, 6.1000]])
Loss = 0.07888662070035934


## The Encoder module.

In [8]:
class Encoder(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, output_embed_dim):
        super().__init__()
        # batch size 32, output 128
        # nn embedding model, integers --> token embeddings
        # 32 X seqLen
        # 32 X seqLeng X512
        # the embeddings output 128
        self.embedding_layer = torch.nn.Embedding(vocab_size, embed_dim)
        self.encoder = torch.nn.TransformerEncoder(
            torch.nn.TransformerEncoderLayer(embed_dim, nhead=8, batch_first=True),
            num_layers=3,
            norm=torch.nn.LayerNorm([embed_dim]),
            enable_nested_tensor=False
        )
        self.projection = torch.nn.Linear(embed_dim, output_embed_dim)

    def forward(self, tokenizer_output):
        x = self.embedding_layer(tokenizer_output['input_ids'])
        x = self.encoder(x, src_key_padding_mask=tokenizer_output['attention_mask'].logical_not())
        cls_embed = x[:,0,:]
        return self.projection(cls_embed)

![Diagram 1](DLAI-diagram-2.png)

## Training Loop

In [9]:
def train_loop(dataset):
    embed_size = 512
    output_embed_size = 128
    max_seq_len = 64
    batch_size = 32

    # define the question/answer encoders
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    question_encoder = Encoder(tokenizer.vocab_size, embed_size,
                               output_embed_size)
    answer_encoder = Encoder(tokenizer.vocab_size, embed_size,
                             output_embed_size)

    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                             shuffle=True)
    # cross entropy
    optimizer = torch.optim.Adam(
        list(question_encoder.parameters()) + list(answer_encoder.parameters()
    ), lr=1e-5)
    loss_fn = torch.nn.CrossEntropyLoss()

    running_loss = []
    for _, data_batch in enumerate(dataloader):

        # Tokenize the question/answer pairs (each is a batch of 32 questions and 32 answers)
        question, answer = data_batch
        question_tok = tokenizer(question, padding=True, truncation=True, return_tensors='pt', max_length=max_seq_len)
        answer_tok = tokenizer(answer, padding=True, truncation=True, return_tensors='pt', max_length=max_seq_len)

        # Compute the embeddings: the output is of dim = 32 x 128
        question_embed = question_encoder(question_tok)
        answer_embed = answer_encoder(answer_tok)

        # Compute similarity scores: a 32x32 matrix
        # row[N] reflects similarity between question[N] and answers[0...31]
        similarity_scores = question_embed @ answer_embed.T

        # we want to maximize the values in the diagonal
        target = torch.arange(question_embed.shape[0], dtype=torch.long)
        loss = loss_fn(similarity_scores, target)
        running_loss += [loss.item()]

        # this is where the magic happens
        optimizer.zero_grad()    # reset optimizer so gradients are all-zero
        loss.backward()
        optimizer.step()

    return question_encoder, answer_encoder

## Training in multiple Epochs

In [10]:
# same setup, but make it a function so we can reuse it

def train(dataset, num_epochs=10):
    embed_size = 512
    output_embed_size = 128
    max_seq_len = 64
    batch_size = 32

    n_iters = len(dataset) // batch_size + 1

    # define the question/answer encoders
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    question_encoder = Encoder(tokenizer.vocab_size, embed_size, output_embed_size)
    answer_encoder = Encoder(tokenizer.vocab_size, embed_size, output_embed_size)

    # define the dataloader, optimizer and loss function
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.Adam(list(question_encoder.parameters()) + list(answer_encoder.parameters()), lr=1e-5)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        running_loss = []
        for idx, data_batch in enumerate(dataloader):

            # Tokenize the question/answer pairs (each is a batc of 32 questions and 32 answers)
            question, answer = data_batch
            question_tok = tokenizer(question, padding=True, truncation=True, return_tensors='pt', max_length=max_seq_len)
            answer_tok = tokenizer(answer, padding=True, truncation=True, return_tensors='pt', max_length=max_seq_len)
            if inx == 0 and epoch == 0:
                print(question_tok['input_ids'].shape, answer_tok['input_ids'].shape)

            # Compute the embeddings: the output is of dim = 32 x 128
            question_embed = question_encoder(question_tok)
            answer_embed = answer_encoder(answer_tok)
            if inx == 0 and epoch == 0:
                print(question_embed.shape, answer_embed.shape)

            # Compute similarity scores: a 32x32 matrix
            # row[N] reflects similarity between question[N] and answers[0...31]
            similarity_scores = question_embed @ answer_embed.T
            if inx == 0 and epoch == 0:
                print(similarity_scores.shape)

            # we want to maximize the values in the diagonal
            target = torch.arange(question_embed.shape[0], dtype=torch.long)
            loss = loss_fn(similarity_scores, target)
            running_loss += [loss.item()]
            if idx == n_iters-1:
                print(f"Epoch {epoch}, loss = ", np.mean(running_loss))

            # this is where the magic happens
            optimizer.zero_grad()    # reset optimizer so gradients are all-zero
            loss.backward()
            optimizer.step()

    return question_encoder, answer_encoder

## Let's train

In [12]:
# answer and question pairs

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, datapath):
        self.data = pd.read_csv(datapath, sep="\t", nrows=300)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data.iloc[idx]['questions'], self.data.iloc[idx]['answers']

# we just chose this dataset
dataset = MyDataset('./shared_data/nq_sample.tsv')
dataset.data.head(5)

,questions,answers
0,who played bubba in the tv series in the heat ...,Carlos Alan Autry Jr. (also known for a period...
1,where did the 2017 tour de france start,"The 3,540 km (2,200 mi)-long race commenced wi..."
2,who is the chess champion of the world,Current world champion Magnus Carlsen won the ...
3,who scored the most hat tricks in football,Cristiano Ronaldo and Messi have scored three ...
4,what do you need to be an ontario scholar,Ontario Scholars are high school graduates in ...


<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(num_epochs = 5)</code>:</b> The <code>num_epochs</code> is set to <code>5</code> for speedier execution. You may train the model
using a higher number of epochs by changing this parameter.</p>

In [18]:
qe, ae = train(dataset, num_epochs=50)

# just show, could be very larg epochs.
# with 50 epochs, you still do not get the correct answer
# loss from 3.685177206993103 to 0.0633135100826621

Epoch 0, loss =  3.685177206993103
Epoch 1, loss =  3.559368062019348
Epoch 2, loss =  3.3996769428253173
Epoch 3, loss =  3.3536009550094605
Epoch 4, loss =  3.3046725034713744
Epoch 5, loss =  3.217980480194092
Epoch 6, loss =  3.2009748220443726
Epoch 7, loss =  3.1478068351745607
Epoch 8, loss =  3.104023313522339
Epoch 9, loss =  2.9822537660598756
Epoch 10, loss =  2.9273232936859133
Epoch 11, loss =  2.851041853427887
Epoch 12, loss =  2.7698217630386353
Epoch 13, loss =  2.6535635113716127
Epoch 14, loss =  2.489632260799408
Epoch 15, loss =  2.4132778644561768
Epoch 16, loss =  2.3328142523765565
Epoch 17, loss =  2.2107775092124937
Epoch 18, loss =  1.9972907423973083
Epoch 19, loss =  1.9028201580047608
Epoch 20, loss =  1.6866387724876404
Epoch 21, loss =  1.5735480070114136
Epoch 22, loss =  1.443335384130478
Epoch 23, loss =  1.2469069123268128
Epoch 24, loss =  1.03388711810112
Epoch 25, loss =  0.9558795899152756
Epoch 26, loss =  0.8536964371800423
Epoch 27, loss =  0.

In [19]:
question = 'What is the tallest mountain in the world?'
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
question_tok = tokenizer(question, padding=True, truncation=True, return_tensors='pt', max_length=64)
question_emb = qe(question_tok)[0]
print(question_tok)
print(question_emb[:5])
print(question)
# just to see the input integers and embeddings would be different.

{'input_ids': tensor([[  101,  2054,  2003,  1996, 13747,  3137,  1999,  1996,  2088,  1029,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([-1.4179,  0.3797,  0.1785,  0.0984,  0.1167], grad_fn=<SliceBackward0>)
What is the tallest mountain in the world?


In [20]:
answers = [
    "What is the tallest mountain in the world?",
    "The tallest mountain in the world is Mount Everest.",
    "Who is donald duck?"
]
answer_tok = []
answer_emb = []
for answer in answers:
    tok = tokenizer(answer, padding=True, truncation=True, return_tensors='pt', max_length=64)
    answer_tok.append(tok['input_ids'])
    emb = ae(tok)[0]
    answer_emb.append(emb)

print(answer_tok)
print(answer_emb[0][:5])
print(answer_emb[1][:5])
print(answer_emb[2][:5])
print(answer)

[tensor([[  101,  2054,  2003,  1996, 13747,  3137,  1999,  1996,  2088,  1029,
           102]]), tensor([[  101,  1996, 13747,  3137,  1999,  1996,  2088,  2003,  4057, 23914,
          1012,   102]]), tensor([[ 101, 2040, 2003, 6221, 9457, 1029,  102]])]
tensor([-0.1007, -0.7156, -0.5800,  0.5016, -1.0491], grad_fn=<SliceBackward0>)
tensor([-0.0444, -0.9409, -0.4757,  0.2325, -0.6030], grad_fn=<SliceBackward0>)
tensor([-0.3486, -0.1412, -1.0564,  0.8505, -0.3759], grad_fn=<SliceBackward0>)
Who is donald duck?


In [21]:
question_emb @ torch.stack(answer_emb).T

tensor([ 3.8581,  1.7138, -2.3971], grad_fn=<SqueezeBackward4>)

In [22]:
question = 'What is the best type of noodles?'
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
question_tok = tokenizer(question, padding=True, truncation=True, return_tensors='pt', max_length=64)
question_emb = qe(question_tok)[0]
print(question_tok)
print(question_emb[:5])

{'input_ids': tensor([[  101,  2054,  2003,  1996,  2190,  2828,  1997, 27130,  1029,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([-0.1385,  0.5270, -0.1360,  0.0173,  0.4712], grad_fn=<SliceBackward0>)


In [23]:
answers = [
    'What is the best type of noodles?',
    "the best type of noodle is jelly noodle",
    "Who is donald duck?"
]
answer_tok = []
answer_emb = []
for answer in answers:
    tok = tokenizer(answer, padding=True, truncation=True, return_tensors='pt', max_length=64)
    answer_tok.append(tok['input_ids'])
    emb = ae(tok)[0]
    answer_emb.append(emb)

print(answer_tok)
print(answer_emb[0][:5])
print(answer_emb[1][:5])
print(answer_emb[2][:5])
print(answer)

[tensor([[  101,  2054,  2003,  1996,  2190,  2828,  1997, 27130,  1029,   102]]), tensor([[  101,  1996,  2190,  2828,  1997,  2053, 26156,  2003, 20919,  2053,
         26156,   102]]), tensor([[ 101, 2040, 2003, 6221, 9457, 1029,  102]])]
tensor([-0.4858, -0.2156, -0.9164,  0.7267, -0.1336], grad_fn=<SliceBackward0>)
tensor([-0.6265,  0.0294, -0.6781,  0.5177, -0.0026], grad_fn=<SliceBackward0>)
tensor([-0.1988,  0.1506, -1.0794,  0.7419, -0.6434], grad_fn=<SliceBackward0>)
Who is donald duck?


In [24]:
question_emb @ torch.stack(answer_emb).T

tensor([15.2859, 14.3123,  8.8230], grad_fn=<SqueezeBackward4>)

# the training is built through PyTorch modules,
# and we see how the training develops
# one example with small example.
# how this works well with full trained algo that can work with RAG in next lesson